In [2]:
import numpy as np

In [1]:
def extract_buried_points(subunit_coords, complex_coords, threshold):

    '''Function that takes as input the coordinates of two similar pointclouds (one subunit and
    one complex involving that subunit) and a threshold. Returns a list of indeces indicating 
    which points of the first pointcloud (subunit) are more than an certain distance (threshold)
    away from any point in the second pointcloud. These points are "buried" in the complex.'''

    from sklearn.neighbors import NearestNeighbors
    
    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(complex_coords) 

    buried= []
    largest_dist = 0
    for i in range(len(subunit_coords)):
        if (dist := neigh.kneighbors([subunit_coords[i]])[0][0][0]) > threshold:
        #if neigh.kneighbors([subunit_coords[i]])[0][0][0] > threshold:
            buried.append(i)
            if dist > largest_dist:
                largest_dist = dist
    return buried 

### Extract buried region and save data in protein object

In [3]:
import numpy as np
import os
from helper_functions import load_object, save_object
from c_ProteinGraph import ProteinGraph
from sklearn.neighbors import NearestNeighbors

path = 'C:/Users/david/pyproj/pyg/adl/surfaces_graphs/'
path_list = [file[0:-4] for file in os.listdir(path)]

dest_dir = 'C:/Users/david/OneDrive - ZHAW/ProteinSurfaces/surfaces_graphs_iface/'

threshold = 1.5

complex_files_lists = list(np.load('complexes_names.npy', allow_pickle=True))

for i, complex_list in enumerate(complex_files_lists):
    if complex_list[0][0:-4] not in path_list: continue
    subunit1 = load_object(path + complex_list[0][0:-3] + 'pkl')

    if complex_list[1][0:-4] not in path_list: continue
    subunit2 = load_object(path + complex_list[1][0:-3] + 'pkl')

    if complex_list[2][0:-4] not in path_list: continue
    complex = load_object(path + complex_list[2][0:-3] + 'pkl')

    subunit1_iface_idx = extract_buried_points(subunit1.pos, complex.pos, threshold)
    subunit2_iface_idx = extract_buried_points(subunit2.pos, complex.pos, threshold)


    ## If the interface is too large, skip this complex
    if len(subunit1_iface_idx) > 300 or len(subunit2_iface_idx) > 300: 
        continue

    ## If the interface is too small, skip this complex
    if len(subunit1_iface_idx) < 50 or len(subunit2_iface_idx) < 50: 
        continue

    subunit1_iface_pos = subunit1.pos[subunit1_iface_idx]
    subunit2_iface_pos = subunit2.pos[subunit2_iface_idx]

    merged_iface = np.concatenate((subunit1_iface_pos, subunit2_iface_pos))
    mean_iface_pos = np.mean(merged_iface, axis = 0)

    neigh1 = NearestNeighbors(n_neighbors=1)

    neigh1.fit(subunit1_iface_pos)
    ix = neigh1.kneighbors([mean_iface_pos], return_distance=False)[0][0]
    subunit1_center_idx = subunit1_iface_idx[ix]
    subunit1_center_pos = subunit1_iface_pos[ix]

    neigh1.fit(subunit2_iface_pos)
    ix = neigh1.kneighbors([mean_iface_pos], return_distance=False)[0][0]
    subunit2_center_idx = subunit2_iface_idx[ix]
    subunit2_center_pos = subunit2_iface_pos[ix]

    # save the new data in the protein objects

    subunit1.iface_idx = subunit1_iface_idx
    subunit1.iface_pos = subunit1_iface_pos
    subunit1.iface_center = (subunit1_center_idx, subunit1_center_pos)

    subunit2.iface_idx = subunit2_iface_idx
    subunit2.iface_pos = subunit2_iface_pos
    subunit2.iface_center = (subunit2_center_idx, subunit2_center_pos)

    save_object(subunit1, dest_dir + complex_list[0][0:-3] + 'pkl')
    print(f'Protein {complex_list[0][0:-3]} saved')
    save_object(subunit2, dest_dir + complex_list[1][0:-3] + 'pkl')
    print(f'Protein {complex_list[1][0:-3]} saved')

Protein 1A99_D. saved
Protein 1A99_C. saved
Protein 1ACB_I. saved
Protein 1ACB_E. saved
Protein 1BCP_L. saved
Protein 1BCP_H. saved
Protein 1BK6_E. saved
Protein 1BK6_B. saved
Protein 1BVK_F. saved
Protein 1BVK_DE. saved
Protein 1C8N_C. saved
Protein 1C8N_A. saved
Protein 1CBW_D. saved
Protein 1CBW_ABC. saved
Protein 1D2Z_B. saved
Protein 1D2Z_A. saved
Protein 1DBW_B. saved
Protein 1DBW_A. saved
Protein 1DFJ_I. saved
Protein 1DFJ_E. saved
Protein 1DK4_B. saved
Protein 1DK4_A. saved
Protein 1DM5_F. saved
Protein 1DM5_E. saved
Protein 1DSX_H. saved
Protein 1DSX_E. saved
Protein 1EAI_C. saved
Protein 1EAI_A. saved
Protein 1EJA_B. saved
Protein 1EJA_A. saved
Protein 1EMU_B. saved
Protein 1EMU_A. saved
Protein 1EMV_B. saved
Protein 1EMV_A. saved
Protein 1EZI_B. saved
Protein 1EZI_A. saved
Protein 1F5R_I. saved
Protein 1F5R_A. saved
Protein 1FGL_B. saved
Protein 1FGL_A. saved
Protein 1FMO_I. saved
Protein 1FMO_E. saved
Protein 1FNS_A. saved
Protein 1FNS_HL. saved
Protein 1FQJ_B. saved
Protei

### Check what there is in the directory and visualize

In [3]:
dest_dir = 'C:/Users/david/pyproj/pyg/adl/surfaces_graphs_iface/'
list_dest_dir = os.listdir(dest_dir)

complex_names = [] 
for name in os.listdir(dest_dir):
    if name[0:4] not in complex_names:
        complex_names.append(name[0:4])

In [24]:
### Import the two files corresponding to that complex

complex = complex_names[3]
print(complex)

import fnmatch
complex_filenames = fnmatch.filter(list_dest_dir, complex+'*')
print(complex_filenames)

1BK6
['1BK6_B.pkl', '1BK6_E.pkl']


In [25]:
from helper_functions import load_object

subunit1 = load_object(dest_dir + complex_filenames[0])
subunit2 = load_object(dest_dir + complex_filenames[1])

In [26]:
subunit1.iface_center

(4833, array([57.1716, 35.6882, 22.4021], dtype=float32))

In [27]:
subunit1.iface_idx

[47,
 53,
 319,
 396,
 414,
 484,
 544,
 624,
 653,
 660,
 668,
 735,
 810,
 951,
 970,
 995,
 1000,
 1096,
 1235,
 1254,
 1265,
 1280,
 1298,
 1376,
 1383,
 1462,
 1554,
 1598,
 1652,
 1754,
 1847,
 1904,
 1977,
 2058,
 2091,
 2096,
 2148,
 2240,
 2252,
 2254,
 2303,
 2531,
 2582,
 2885,
 2921,
 2958,
 3027,
 3052,
 3091,
 3110,
 3162,
 3199,
 3228,
 3284,
 3313,
 3315,
 3324,
 3397,
 3403,
 3435,
 3456,
 3584,
 3625,
 3638,
 3656,
 3731,
 3741,
 3744,
 3858,
 3941,
 3954,
 3998,
 4111,
 4129,
 4181,
 4248,
 4326,
 4333,
 4399,
 4427,
 4447,
 4533,
 4555,
 4573,
 4668,
 4673,
 4682,
 4697,
 4818,
 4833,
 4908,
 5038,
 5052,
 5112,
 5134,
 5190,
 5191,
 5354,
 5363,
 5404,
 5485,
 5512,
 5582,
 5626,
 5627,
 5635,
 5642,
 5682,
 5689,
 5861,
 5886,
 5921,
 6070,
 6104,
 6186,
 6207,
 6356,
 6393,
 6450,
 6454,
 6530,
 6572,
 6606,
 6617,
 6676,
 6710,
 6711,
 6717,
 6775,
 6827,
 6860,
 6902,
 6918,
 6947,
 6958,
 6983,
 7018,
 7038,
 7066,
 7133,
 7235,
 7253,
 7290]

In [28]:
subunit1.iface_pos.shape

(143, 3)

### Visualize subunit 1

In [29]:
from f_visualize_pointcloud import visualize_pointcloud

colors_subunit1 = np.zeros_like(subunit1.pos)
colors_subunit1[:] = [1,0,0]
colors_subunit1[subunit1.iface_idx] = [0,0,1]
colors_subunit1[subunit1.iface_center[0]] = [0,1,0]

visualize_pointcloud(subunit1.pos, colors = colors_subunit1)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


### Visualize Subunit 2

In [30]:
from f_visualize_pointcloud import visualize_pointcloud

colors_subunit2 = np.zeros_like(subunit2.pos)
colors_subunit2[:] = [1,0,0]
colors_subunit2[subunit2.iface_idx] = [0,0,1]
colors_subunit2[subunit2.iface_center[0]] = [0,1,0]

visualize_pointcloud(subunit2.pos, colors = colors_subunit2)

### Visualize the complex

In [31]:
merged_coords = np.concatenate((subunit1.pos, subunit2.pos))
merged_colors = np.concatenate((colors_subunit1, colors_subunit2))

In [32]:
visualize_pointcloud(merged_coords, colors = merged_colors)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
